In [1]:
from pyomo.environ import *
import math
from pyomo.environ import exp
import os
# gams_path = r'C:\GAMS\45'
# os.environ["PATH"] += os.pathsep + gams_path
# import sys
# sys.path.append(r'C:\GAMS\45')

In [2]:
m = ConcreteModel()

# Design variables
m.d0 = Var(bounds=(0, 5))
m.d1 = Var(bounds=(0, 5))

# Binary variables indicating region membership
m.y0 = Var(within=Binary)
m.y1 = Var(within=Binary)
m.y2 = Var(within=Binary)
m.y3 = Var(within=Binary)
m.y4 = Var(within=Binary)
m.y5 = Var(within=Binary)

m.sf = Var(within=NonNegativeReals)

M = 1000
target = 0.65

# Region 0:
m.r0c0 = Constraint(expr = m.d0 -2*m.d1 -5/3 <= M*(1-m.y0))
m.r0c1 = Constraint(expr = m.d0 -2*m.d1 -13/3 <= M*(1-m.y0))
m.r0c2 = Constraint(expr = -m.d0 +2*m.d1 +7/3 <= M*(1-m.y0))

# Region 1:
m.r1c0 = Constraint(expr = m.d0 -2*m.d1 -5/3 <= M*(1-m.y1))
m.r1c1 = Constraint(expr = -4*m.d0 +4*m.d1 +8/3 <= M*(1-m.y1))
m.r1c2 = Constraint(expr = m.d0 -2*m.d1 -7/3 <= M*(1-m.y1))

# Region 2:
m.r2c0 = Constraint(expr = m.d0 -2*m.d1 -5/3 <= M*(1-m.y2))
m.r2c1 = Constraint(expr = m.d0 -m.d1 -2/3 <= M*(1-m.y2))

# Region 3:
m.r3c0 = Constraint(expr = -m.d0 +2*m.d1 -5/3 <= M*(1-m.y3))
m.r3c1 = Constraint(expr = m.d0 -2*m.d1 -7 <= M*(1-m.y3))
m.r3c2 = Constraint(expr = -m.d0 +2*m.d1 +3 <= M*(1-m.y3))

# Region 4:
m.r4c0 = Constraint(expr = -m.d0 +2*m.d1 -5/3 <= M*(1-m.y4))
m.r4c1 = Constraint(expr = -3*m.d0 +2*m.d1 +1 <= M*(1-m.y4))
m.r4c2 = Constraint(expr = m.d0 - 2*m.d1 - 3 <= M*(1-m.y4))

# Region 5:
m.r5c0 = Constraint(expr = -m.d0 +2*m.d1 -5/3 <= M*(1-m.y5))
m.r5c1 = Constraint(expr = 3*m.d0 -2*m.d1 -1 <= M*(1-m.y5))

# Only one region active
m.only_one_region = Constraint(expr=m.y0 + m.y1 + m.y2 + m.y3 + m.y4 + m.y5 == 1)

# SF expressions
sf0 = (-16*m.d0 + 32*m.d1 + 208/3)*exp(-2*(-m.d0 + 2*m.d1 + 7/3)**2)/math.pi
sf1 = 32/math.pi
sf2 = (4*m.d0 -4*m.d1 + 88/3)*exp(-2*(-0.25*m.d0 +0.25*m.d1 +1/6)**2)/math.pi
sf3 = 8*(-m.d0 + 2*m.d1 +7)*(-0.1875*m.d0 +0.375*m.d1 +1.3125)*exp(-2*(0.375*m.d0 -0.75*m.d1 -0.625)**2 -2*(0.5*m.d0 -m.d1 -1.5)**2)/math.pi
sf4 = (-6*m.d0 +12*m.d1 +42)*exp(-2*(-0.375*m.d0 +0.75*m.d1 +0.625)**2)/math.pi
sf5 = 8*(-0.1875*m.d0 +0.375*m.d1 +1.3125)*(0.375*m.d0 -0.25*m.d1 +3.875)*exp(-2*(-0.1875*m.d0 +0.125*m.d1 +0.0625)**2 -2*(0.375*m.d0 -0.75*m.d1 -0.625)**2)/math.pi


# # Conditional constraints
m.sf0_con = Constraint(expr=sf0 >= target - M*(1 - m.y0))
m.sf1_con = Constraint(expr=sf1 >= target - M*(1 - m.y1))
m.sf2_con = Constraint(expr=sf2 >= target - M*(1 - m.y2))
m.sf3_con = Constraint(expr=sf3 >= target - M*(1 - m.y3))
m.sf4_con = Constraint(expr=sf4 >= target - M*(1 - m.y4))
m.sf5_con = Constraint(expr=sf5 >= target - M*(1 - m.y5))

m.sf_con = Constraint(expr=m.sf == sf0*m.y0 + sf1*m.y1 + sf2*m.y2 + sf3*m.y3 + sf4*m.y4 + sf5*m.y5)
m.obj = Objective(expr=10*m.d0 + 10*m.d1, sense=minimize)

In [3]:
m.sf0_con.pprint()

sf0_con : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                        : Upper : Active
    None :  -Inf : 0.65 - 1000*(1 - y0) - (-16*d0 + 32*d1 + 69.33333333333333)*exp(-2*(- d0 + 2*d1 + 2.3333333333333335)**2)/3.141592653589793 :   0.0 :   True


In [4]:
results = SolverFactory('gams', solver='baron').solve(m, tee=True)
# results = SolverFactory('gurobi', solver_io = 'python').solve(m, tee=True)

--- Job model.gms Start 07/10/25 11:39:08 45.7.0 64fbf3ce WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\45\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpmjis_2bk\model.gms
    Output C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpmjis_2bk\output.lst
    ScrDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpmjis_2bk\225a\
    SysDir C:\GAMS\45\
    CurDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpmjis_2bk\
    LogOption 3
Licensee: MUD - 30 User License                          G230830|0002AO-GEN
          Texas A&M University, Chemical Engineering                DC11194
          C:\GAMS\45\gamslice.txt
          License Admin: Jeff Polasek, j-polasek@tamu.edu                  
          The maintenance period of the license expired on Jun 25, 2024
          Please contact GAMS or your distributor for further information
Processor information: 1 socket(s), 16 core(s), and 24 thread(s) available
GAMS 45.7.0   Copyright (C) 1987-2024 

In [5]:
m.y2.pprint()

y2 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   1.0 :     1 : False : False : Binary


In [6]:
m.d0.pprint()

d0 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :     5 : False : False :  Reals


In [7]:
m.d1.pprint()

d1 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :     5 : False : False :  Reals


In [8]:
m.sf.pprint()

sf : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 8.832508692544286 :  None : False : False : NonNegativeReals


In [9]:
m.pprint()

9 Var Declarations
    d0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     5 : False : False :  Reals
    d1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     5 : False : False :  Reals
    sf : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 8.832508692544286 :  None : False : False : NonNegativeReals
    y0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    y3 : Size=1, Index=None
       